In [4]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist

import skimage.feature
import numpy as np
import time

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import backend as K
from keras import metrics

#has to do some operations with tensorflow
import tensorflow as tf

#batch_size
BATCH_SIZE = 100

class GLCM(Layer):


    def __init__(self, BATCH_SIZE = 100, d = [1], theta = [0], levels = 256, symmetric = True, normed = True, **kwargs ):
        super(GLCM, self).__init__(**kwargs)
        self.d = d
        self.theta = theta
        self.levels = levels
        self.symmetric = symmetric
        self.normed = normed
        self.batch_size = BATCH_SIZE

    def compute_output_shape(self, input_shape):

        return (self.batch_size, 4)

    def binarization(self, x):
        bi_x = K.sign(x)
        return bi_x
    
    def call(self, inputs):
        #make input becomes a binarized image
        image = self.binarization(inputs)

        #calculate GLCM(in a batch shape)
        a = image[:, :, :-1, :]
        b = image[:, :, 1:, :]

        a_or_b = a+b
        a_and_b = a*b

        one = tf.ones(shape = a.shape)

        not_a = one-a
        not_b = one-b

        # not(a and b)
        M0_0 = K.map_fn(K.sum, (one-a_and_b))

        # b and (not a)
        M0_1 = K.map_fn(K.sum, (b*(not_a)))

        # a and (not b)
        M1_0 = K.map_fn(K.sum, (a*(not_b)))

        # a and b
        M1_1 = K.map_fn(K.sum, (a_and_b))


        output = tf.stack([M0_0, M0_1, M1_0, M1_1], axis = 1)

        
        #print("output shape: ", output.shape)
        
        return output

In [6]:
x = Input(batch_shape=(BATCH_SIZE,28, 28, 1))
y = GLCM()(x)
z = Dense(10, activation='softmax')(y)
model = Model(x, z)
model.compile(optimizer='Adam', loss="MSE", metrics=['acc'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (100, 28, 28, 1)          0         
_________________________________________________________________
glcm_2 (GLCM)                (100, 4)                  0         
_________________________________________________________________
dense_2 (Dense)              (100, 10)                 50        
Total params: 50
Trainable params: 50
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(x_train, y_train,
          batch_size= BATCH_SIZE,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 7s - loss: 0.1741 - acc: 0.1209 - val_loss: 0.1727 - val_acc: 0.1269
Epoch 2/12
60000/60000 [==============================] - 7s - loss: 0.1726 - acc: 0.1282 - val_loss: 0.1726 - val_acc: 0.1283
Epoch 3/12
60000/60000 [==============================] - 7s - loss: 0.1716 - acc: 0.1329 - val_loss: 0.1717 - val_acc: 0.1329
Epoch 4/12
60000/60000 [==============================] - 7s - loss: 0.1704 - acc: 0.1367 - val_loss: 0.1689 - val_acc: 0.1441
Epoch 5/12
60000/60000 [==============================] - 8s - loss: 0.1680 - acc: 0.1466 - val_loss: 0.1647 - val_acc: 0.1598
Epoch 6/12
60000/60000 [==============================] - 8s - loss: 0.1632 - acc: 0.1643 - val_loss: 0.1602 - val_acc: 0.1746
Epoch 7/12
60000/60000 [==============================] - 8s - loss: 0.1609 - acc: 0.1682 - val_loss: 0.1602 - val_acc: 0.1759
Epoch 8/12
60000/60000 [==============================] - 8s 